In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Resnet152


In [4]:
from keras.applications.resnet_v2 import ResNet152V2
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.utils import Sequence
import math
import numpy as np

# ResNet152 모델 로드 (최상위 층 제외)
base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 두 데이터 제너레이터를 결합하는 클래스
class CombinedDataGenerator(Sequence):
    def __init__(self, generator1, generator2):
        self.generator1 = generator1
        self.generator2 = generator2
        self.samples = generator1.samples + generator2.samples
        self.total_len = len(generator1) + len(generator2)

    def __len__(self):
        return self.total_len

    def __getitem__(self, idx):
        half_len = self.total_len // 2
        if idx < half_len:
            return self.generator1[idx]
        else:
            new_idx = idx - half_len
            if new_idx < len(self.generator2):
                return self.generator2[new_idx]
            else:
                return self.generator2[len(self.generator2) - 1]

# ImageDataGenerator 인스턴스 생성
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 트레이닝 데이터셋 로드 및 설정
train_generator1 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator2 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 검증 데이터셋 로드 및 설정
validation_generator1 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator2 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 트레이닝 및 검증 데이터 제너레이터 결합
combined_train_generator = CombinedDataGenerator(train_generator1, train_generator2)
combined_validation_generator = CombinedDataGenerator(validation_generator1, validation_generator2)

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = combined_train_generator.samples // train_generator1.batch_size
if combined_train_generator.samples % train_generator1.batch_size > 0:
    steps_per_epoch += 1

validation_steps = combined_validation_generator.samples // validation_generator1.batch_size
if combined_validation_generator.samples % validation_generator1.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    combined_train_generator,
    steps_per_epoch=len(combined_train_generator),
    epochs=20,
    validation_data=combined_validation_generator,
    validation_steps=len(combined_validation_generator),
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('Resnet152_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

Found 1502 images belonging to 2 classes.
Found 1493 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Epoch 1/20
188/188 [==============================] - 158s 420ms/step - loss: 5.5887 - accuracy: 0.4938 - val_loss: 5.4462 - val_accuracy: 0.4422 - lr: 0.0000e+00
Epoch 2/20
188/188 [==============================] - 33s 174ms/step - loss: 5.0228 - accuracy: 0.7369 - val_loss: 5.0443 - val_accuracy: 0.6776 - lr: 2.0000e-05
Epoch 3/20
188/188 [==============================] - 33s 174ms/step - loss: 4.3016 - accuracy: 0.9439 - val_loss: 4.6740 - val_accuracy: 0.7265 - lr: 4.0000e-05
Epoch 4/20
188/188 [==============================] - 33s 175ms/step - loss: 3.8050 - accuracy: 0.9803 - val_loss: 4.3874 - val_accuracy: 0.7156 - lr: 6.0000e-05
Epoch 5/20
188/188 [==============================] - 33s 175ms/step - loss: 3.3299 - accuracy: 0.9880 - val_loss: 3.9616 - val_accuracy: 0.7347 - lr: 8.0000e-05
Epoch 6/20
188/188 [===

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Average Validation Accuracy: 0.7501889632807838


## Mobilenet_V3

In [6]:
from keras.applications.densenet import DenseNet201
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np
from keras.applications import MobileNetV3Large

# MobileNetV3Large 모델 로드 (최상위 층 제외)
base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 두 데이터 제너레이터를 결합하는 클래스
class CombinedDataGenerator(Sequence):
    def __init__(self, generator1, generator2):
        self.generator1 = generator1
        self.generator2 = generator2
        self.samples = generator1.samples + generator2.samples
        self.total_len = len(generator1) + len(generator2)

    def __len__(self):
        return self.total_len

    def __getitem__(self, idx):
        half_len = self.total_len // 2
        if idx < half_len:
            return self.generator1[idx]
        else:
            new_idx = idx - half_len
            if new_idx < len(self.generator2):
                return self.generator2[new_idx]
            else:
                return self.generator2[len(self.generator2) - 1]

# ImageDataGenerator 인스턴스 생성
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 트레이닝 데이터셋 로드 및 설정
train_generator1 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator2 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 검증 데이터셋 로드 및 설정
validation_generator1 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator2 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 트레이닝 및 검증 데이터 제너레이터 결합
combined_train_generator = CombinedDataGenerator(train_generator1, train_generator2)
combined_validation_generator = CombinedDataGenerator(validation_generator1, validation_generator2)

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = combined_train_generator.samples // train_generator1.batch_size
if combined_train_generator.samples % train_generator1.batch_size > 0:
    steps_per_epoch += 1

validation_steps = combined_validation_generator.samples // validation_generator1.batch_size
if combined_validation_generator.samples % validation_generator1.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    combined_train_generator,
    steps_per_epoch=len(combined_train_generator),
    epochs=20,
    validation_data=combined_validation_generator,
    validation_steps=len(combined_validation_generator),
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('Mobilenet_v3_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

Found 1502 images belonging to 2 classes.
Found 1493 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Epoch 1/20
188/188 [==============================] - 42s 97ms/step - loss: 5.0344 - accuracy: 0.5249 - val_loss: 4.9319 - val_accuracy: 0.4490 - lr: 0.0000e+00
Epoch 2/20
188/188 [==============================] - 11s 57ms/step - loss: 4.6926 - accuracy: 0.6581 - val_loss: 4.8610 - val_accuracy: 0.4490 - lr: 2.0000e-05
Epoch 3/20
188/188 [==============================] - 11s 59ms/step - loss: 4.2493 - accuracy: 0.8461 - val_loss: 4.5070 - val_accuracy: 0.4490 - lr: 4.0000e-05
Epoch 4/20
188/188 [==============================] - 11s 59ms/step - loss: 3.8929 - accuracy: 0.9185 - val_loss: 4.3317 - val_accuracy: 0.3456 - lr: 6.0000e-05
Epoch 5/20
188/188 [==============================] - 11s 58ms/step - loss: 3.5398 - accuracy: 0.9593 - val_loss: 4.2185 - val_accuracy: 0.4490 - lr: 8.0000e-05
Epoch 6/20
188/188 [=========

## Inception_V3

In [7]:
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np

# InceptionV3 모델 로드 (최상위 층 제외)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))  # InceptionV3는 299x299 입력을 사용

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 두 데이터 제너레이터를 결합하는 클래스
class CombinedDataGenerator(Sequence):
    def __init__(self, generator1, generator2):
        self.generator1 = generator1
        self.generator2 = generator2
        self.samples = generator1.samples + generator2.samples
        self.total_len = len(generator1) + len(generator2)

    def __len__(self):
        return self.total_len

    def __getitem__(self, idx):
        half_len = self.total_len // 2
        if idx < half_len:
            return self.generator1[idx]
        else:
            new_idx = idx - half_len
            if new_idx < len(self.generator2):
                return self.generator2[new_idx]
            else:
                return self.generator2[len(self.generator2) - 1]

# ImageDataGenerator 인스턴스 생성
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 트레이닝 데이터셋 로드 및 설정
train_generator1 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator2 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 검증 데이터셋 로드 및 설정
validation_generator1 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator2 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 트레이닝 및 검증 데이터 제너레이터 결합
combined_train_generator = CombinedDataGenerator(train_generator1, train_generator2)
combined_validation_generator = CombinedDataGenerator(validation_generator1, validation_generator2)

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = combined_train_generator.samples // train_generator1.batch_size
if combined_train_generator.samples % train_generator1.batch_size > 0:
    steps_per_epoch += 1

validation_steps = combined_validation_generator.samples // validation_generator1.batch_size
if combined_validation_generator.samples % validation_generator1.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    combined_train_generator,
    steps_per_epoch=len(combined_train_generator),
    epochs=20,
    validation_data=combined_validation_generator,
    validation_steps=len(combined_validation_generator),
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('InceptionV3_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

87910968/87910968 [==============================] - 3s 0us/step
Found 1502 images belonging to 2 classes.
Found 1493 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Epoch 1/20
188/188 [==============================] - 51s 103ms/step - loss: 5.6161 - accuracy: 0.5072 - val_loss: 5.2831 - val_accuracy: 0.5660 - lr: 0.0000e+00
Epoch 2/20
188/188 [==============================] - 14s 76ms/step - loss: 5.0993 - accuracy: 0.7312 - val_loss: 4.9876 - val_accuracy: 0.7714 - lr: 2.0000e-05
Epoch 3/20
188/188 [==============================] - 14s 75ms/step - loss: 4.5148 - accuracy: 0.9369 - val_loss: 4.8615 - val_accuracy: 0.7524 - lr: 4.0000e-05
Epoch 4/20
188/188 [==============================] - 14s 76ms/step - loss: 4.1726 - accuracy: 0.9760 - val_loss: 4.5161 - val_accuracy: 0.8122 - lr: 6.0000e-05
Epoch 5/20
188/188 [==============================] - 14s 77ms/step - loss: 3.8289 - accuracy: 0.9843 - val_loss: 4.1771 - v

## DenseNet201

In [10]:
from keras.applications.densenet import DenseNet201
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np

# DenseNet201 모델 로드 (최상위 층 제외)
base_model = DenseNet201(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 두 데이터 제너레이터를 결합하는 클래스
class CombinedDataGenerator(Sequence):
    def __init__(self, generator1, generator2):
        self.generator1 = generator1
        self.generator2 = generator2
        self.samples = generator1.samples + generator2.samples
        self.total_len = len(generator1) + len(generator2)

    def __len__(self):
        return self.total_len

    def __getitem__(self, idx):
        half_len = self.total_len // 2
        if idx < half_len:
            return self.generator1[idx]
        else:
            new_idx = idx - half_len
            if new_idx < len(self.generator2):
                return self.generator2[new_idx]
            else:
                return self.generator2[len(self.generator2) - 1]

# ImageDataGenerator 인스턴스 생성
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 트레이닝 데이터셋 로드 및 설정
train_generator1 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator2 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 검증 데이터셋 로드 및 설정
validation_generator1 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator2 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 트레이닝 및 검증 데이터 제너레이터 결합
combined_train_generator = CombinedDataGenerator(train_generator1, train_generator2)
combined_validation_generator = CombinedDataGenerator(validation_generator1, validation_generator2)

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = combined_train_generator.samples // train_generator1.batch_size
if combined_train_generator.samples % train_generator1.batch_size > 0:
    steps_per_epoch += 1

validation_steps = combined_validation_generator.samples // validation_generator1.batch_size
if combined_validation_generator.samples % validation_generator1.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    combined_train_generator,
    steps_per_epoch=len(combined_train_generator),
    epochs=20,
    validation_data=combined_validation_generator,
    validation_steps=len(combined_validation_generator),
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('DenseNet201_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

74836368/74836368 [==============================] - 3s 0us/step
Found 1502 images belonging to 2 classes.
Found 1493 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Epoch 1/20
188/188 [==============================] - 137s 198ms/step - loss: 5.5027 - accuracy: 0.5292 - val_loss: 5.7024 - val_accuracy: 0.5143 - lr: 0.0000e+00
Epoch 2/20
188/188 [==============================] - 29s 153ms/step - loss: 4.7069 - accuracy: 0.7923 - val_loss: 4.8541 - val_accuracy: 0.5687 - lr: 2.0000e-05
Epoch 3/20
188/188 [==============================] - 29s 155ms/step - loss: 3.7182 - accuracy: 0.9593 - val_loss: 4.4147 - val_accuracy: 0.5986 - lr: 4.0000e-05
Epoch 4/20
188/188 [==============================] - 29s 154ms/step - loss: 2.8794 - accuracy: 0.9836 - val_loss: 2.9599 - val_accuracy: 0.8286 - lr: 6.0000e-05
Epoch 5/20
188/188 [==============================] - 29s 154ms/step - loss: 2.2050 - accuracy: 0.9943 - val_loss: 2.184

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Average Validation Accuracy: 0.8509845984609503


## EffcientB7

In [11]:
from keras.applications.efficientnet import EfficientNetB7
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
import math
import numpy as np

# EfficientNetB0 모델 로드 (최상위 층 제외)
base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모든 층을 훈련 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

# 새로운 분류기 층 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer='l2')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # 드롭아웃 비율 조정
predictions = Dense(1, activation='sigmoid')(x)

# 새로운 모델 정의
model = Model(inputs=base_model.input, outputs=predictions)

# 옵티마이저 설정 (Adam 사용)
optimizer = Adam(learning_rate=0.0001)

# 모델 컴파일
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# 두 데이터 제너레이터를 결합하는 클래스
class CombinedDataGenerator(Sequence):
    def __init__(self, generator1, generator2):
        self.generator1 = generator1
        self.generator2 = generator2
        self.samples = generator1.samples + generator2.samples
        self.total_len = len(generator1) + len(generator2)

    def __len__(self):
        return self.total_len

    def __getitem__(self, idx):
        half_len = self.total_len // 2
        if idx < half_len:
            return self.generator1[idx]
        else:
            new_idx = idx - half_len
            if new_idx < len(self.generator2):
                return self.generator2[new_idx]
            else:
                return self.generator2[len(self.generator2) - 1]

# ImageDataGenerator 인스턴스 생성
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# 트레이닝 데이터셋 로드 및 설정
train_generator1 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

train_generator2 = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_TRAIN',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 검증 데이터셋 로드 및 설정
validation_generator1 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER3_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

validation_generator2 = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/DRUNK_DATA/LAYER1_VALID',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary')

# 트레이닝 및 검증 데이터 제너레이터 결합
combined_train_generator = CombinedDataGenerator(train_generator1, train_generator2)
combined_validation_generator = CombinedDataGenerator(validation_generator1, validation_generator2)

# 코사인 니일링 웜업 스케줄러 함수 정의
def cosine_annealing_warmup(epoch, max_epoch=20, base_lr=0.0001, min_lr=0.00001):
    if epoch < 5:  # 웜업 기간 설정 (예: 처음 5 에포크)
        return base_lr * epoch / 5
    else:
        return min_lr + (base_lr - min_lr) * (1 + math.cos(math.pi * (epoch - 5) / (max_epoch - 5))) / 2

# 코사인 니일링 웜업 스케줄러 콜백 생성
cosine_lr_scheduler = LearningRateScheduler(cosine_annealing_warmup)

# 조기 중단 콜백 설정 (정확도 기반)
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=10)

# steps_per_epoch와 validation_steps 계산
steps_per_epoch = combined_train_generator.samples // train_generator1.batch_size
if combined_train_generator.samples % train_generator1.batch_size > 0:
    steps_per_epoch += 1

validation_steps = combined_validation_generator.samples // validation_generator1.batch_size
if combined_validation_generator.samples % validation_generator1.batch_size > 0:
    validation_steps += 1

# 모델 학습
history = model.fit(
    combined_train_generator,
    steps_per_epoch=len(combined_train_generator),
    epochs=20,
    validation_data=combined_validation_generator,
    validation_steps=len(combined_validation_generator),
    callbacks=[cosine_lr_scheduler, early_stopping])

# 모델 저장
model.save('EfficientNetB7_drunk_or_sober.h5')

# 각 epoch에서의 검증 정확도(val_accuracy) 리스트 가져오기
val_accuracies = history.history['val_accuracy']

# 평균 검증 정확도 계산
average_val_accuracy = np.mean(val_accuracies)

print("Average Validation Accuracy:", average_val_accuracy)

258076736/258076736 [==============================] - 8s 0us/step
Found 1502 images belonging to 2 classes.
Found 1493 images belonging to 2 classes.
Found 464 images belonging to 2 classes.
Found 323 images belonging to 2 classes.
Epoch 1/20
188/188 [==============================] - 185s 344ms/step - loss: 5.7057 - accuracy: 0.5082 - val_loss: 5.3485 - val_accuracy: 0.4503 - lr: 0.0000e+00
Epoch 2/20
188/188 [==============================] - 59s 316ms/step - loss: 5.1818 - accuracy: 0.7245 - val_loss: 5.4368 - val_accuracy: 0.5469 - lr: 2.0000e-05
Epoch 3/20
188/188 [==============================] - 59s 315ms/step - loss: 4.6698 - accuracy: 0.9065 - val_loss: 5.1415 - val_accuracy: 0.5823 - lr: 4.0000e-05
Epoch 4/20
188/188 [==============================] - 59s 316ms/step - loss: 4.2846 - accuracy: 0.9623 - val_loss: 4.6109 - val_accuracy: 0.7551 - lr: 6.0000e-05
Epoch 5/20
188/188 [==============================] - 59s 316ms/step - loss: 3.8828 - accuracy: 0.9776 - val_loss: 4.4